# DataJoint Elements for Electrode Localization

#### Open-source data pipeline for localizing Neuropixels electrodes within the Allen brain atlas - [Allen Mouse Common Coordinate Framework (CCF)](https://atlas.brain-map.org/)

Welcome to the tutorial for the DataJoint Element for electrode localization. This tutorial aims to provide a comprehensive understanding of the open-source data pipeline created using `element-electrode-localization`.

![flowchart](../images/flowchart.svg)

This package is designed to seamlessly ingest and track **electrode localization** data using DataJoint. 

![flowchart](../images/pipeline.svg)

By the end of this tutorial, you will have a clear grasp on setting up  `element-electrode-localization` into your specific research projects and lab.

### Prerequisites
Please see the [datajoint tutorials GitHub repository](https://github.com/datajoint/datajoint-tutorials/tree/main) before proceeding.

#### **Tutorial Overview**
+ Setup
+ *Activate* the DataJoint pipeline.
+ *Insert* example data into subject and session tables.
+ *Description* of the main tables.

### **Setup**
This tutorial describes the DataJoint tables that track the electrode localization data acquired with Neuropixels probes. The goal is to store, track, and manage all metadata associated with electrode localization and position probes, aligning to the Common Coordinate Framework (CCF) space within the Allen brain atlas.

The results of this Element can be combined with **other modalities** to create a complete, customizable data pipeline for your specific lab or study. For instance, you can combine `element-electrode-localization` with `element-array-ephys` and `element-event` to characterize the neural activity during specific stimulus events.

Let's start this tutorial by importing the packages necessary to run the notebook.

In [1]:
import datajoint as dj
from pathlib import Path
import yaml

If the tutorial is run in Codespaces, a private, local database server is created and made available for you. This is where we will insert and store our processed results. Let's connect to the database server.

In [2]:
dj.conn()

[2024-01-26 00:56:11,248][INFO]: Connecting root@fakeservices.datajoint.io:3306
[2024-01-26 00:56:11,257][INFO]: Connected root@fakeservices.datajoint.io:3306


DataJoint connection (connected) root@fakeservices.datajoint.io:3306

### **Activate the DataJoint Pipeline**

This tutorial presumes that the `element-electrode-localization` has been pre-configured and instantiated, with the database linked downstream to pre-existing `subject` and `session` tables. Please refer to the [`tutorial_pipeline.py`](./tutorial_pipeline.py) for the source code.

Now, we will proceed to import the essential schemas required to construct this data pipeline, with particular attention to the primary component: `electrode_localization`.

In [3]:
from tutorial_pipeline import (
    subject,
    session,
    probe,
    ephys,
    electrode_localization,
    coordinate_framework as ccf,
)

[2024-01-26 00:56:12,796][WARNING]: lab.Project and related tables will be removed in a future version of Element Lab. Please use the project schema.


We can represent the tables in the `electrode-localization` schema as well as some of the upstream dependencies to `session` and `subject` schemas as a diagram.


In [ ]:
(
    dj.Diagram(subject.Subject)
    + dj.Diagram(session.Session)
    + dj.Diagram(electrode_localization)
    + dj.Diagram(ccf)
)

As evident from the diagram, this data pipeline encompasses tables associated with electrode localization data. A few tables, such as `subject.Subject` and `session.Session`, while important for a complete pipeline, fall outside the scope of the `element-electrode-localization` tutorial, and will therefore, not be explored extensively here. The primary focus of this tutorial will be on the `electrode-localization` schema.

In [ ]:
(
    dj.Diagram(electrode_localization)
    + dj.Diagram(ephys.ProbeInsertion)
    + dj.Diagram(ccf)
)

### **Insert example data into subject and session tables**

Now, let's delve into the `subject.Subject` and `session.Session` tables and include some example data.

In [ ]:
subject.Subject()

Add a new entry for a subject in the `Subject` table:

In [ ]:
ccf.BrainRegionAnnotation.BrainRegion()

In [ ]:
central_thalamus = ccf.BrainRegionAnnotation.retrieve_acronym("CM")
cranial_nerves = ccf.BrainRegionAnnotation.retrieve_acronym("cm")
print(f"CM: {central_thalamus}\ncm: {cranial_nerves}")

In [ ]:
cm_voxels = ccf.BrainRegionAnnotation.Voxel() & f"acronym='{central_thalamus}'"
cm_voxels

In [ ]:
subject.Subject.insert1(
    dict(subject="subject5", subject_birth_date="2023-01-01", sex="U")
)
subject.Subject()

Create session keys and input them into the `Session` table:

In [ ]:
session_key = dict(subject="subject5", session_datetime="2023-01-01 00:00:00")
session.Session.insert1(session_key)
session.Session()

In [ ]:
session.SessionDirectory.insert1(
    dict(**session_key, session_dir="raw/subject5/session1")
)
session.SessionDirectory()

In [ ]:
probe.Probe.insert1(
    dict(probe="714000838", probe_type="neuropixels 1.0 - 3B")
)  # this info could be achieve from neuropixels meta file.
probe.Probe()

In [ ]:
ephys.ProbeInsertion.insert1(
    dict(
        session_key,
        insertion_number=1,
        probe="714000838",
    )
)  # probe, subject, session_datetime needs to follow the restrictions of foreign keys.
ephys.ProbeInsertion()

In [5]:
ccf.CCF()

"ccf_id CCF ID, a.k.a atlas ID",ccf_version Allen CCF Version - e.g. CCFv3,ccf_resolution voxel resolution in micron,ccf_description CCFLabel Description
0,ccf_2017,100.0,Version: ccf_2017 - Voxel resolution (uM): 100 - Volume file: annotation_100.nrrd - Region ontology file: query.csv


In [6]:
%pdb on

Automatic pdb calling has been turned ON


In [7]:
electrode_localization.ElectrodePosition.populate()

DataJointError: fetch1 should only return one tuple. 0 tuples found

> /usr/local/lib/python3.9/site-packages/datajoint/fetch.py(352)__call__()
    350             )
    351             if len(result) != 1:
--> 352                 raise DataJointError(
    353                     "fetch1 should only return one tuple. %d tuples found" % len(result)
    354                 )

*** NameError: name 'get_electrode_localization_dir' is not defined
<class 'datajoint.fetch.key'>
*** NameError: name 'get_electrode_localization_dir' is not defined
*** datajoint.errors.DataJointError: Invalid restriction type <class 'datajoint.fetch.key'>
*** TypeError: get_electrode_localization_dir() missing 1 required positional argument: 'probe_insertion_key'


### **Description of the main tables**

To know what data to insert into the table, we can view its dependencies and attributes using the `.heading` method. `heading` provides an exhaustive list of the table's attributes. This list includes both the attributes declared in this table and any inherited from upstream tables.

`ProbeInsertion` table will contain the data for probe insertion implanted into an animal for a given session. Let's insert some example data into the `ProbeInsertion` table as well:

In [ ]:
ccf.BrainRegionAnnotation.BrainRegion()

In [ ]:
central_thalamus = ccf.BrainRegionAnnotation.retrieve_acronym("CM")
cranial_nerves = ccf.BrainRegionAnnotation.retrieve_acronym("cm")
print(f"CM: {central_thalamus}\ncm: {cranial_nerves}")

In [ ]:
electrode_localization.ProbeInsertion.heading

`ElectrodePosition` table computes the electrode position information for a probe insertion.    

In [ ]:
electrode_localization.ElectrodePosition.heading

### Summary

Following this tutorial, we have:

- Covered the essential description of `element-electrode-localization`
- Learned how to manually insert data into tables

#### Documentation and DataJoint tutorials

+ Detailed [documentation on `element-electrode-localization`](https://datajoint.com/docs/elements/element-electrode-localization/0.1/)
+  [General `DataJoint-Python` interactive tutorials](https://github.com/datajoint/datajoint-tutorials), covering fundamentals, such as table tiers, query operations, fetch operations, automated computations with the make function, and more.
+ [Documentation for `DataJoint-Python`](https://datajoint.com/docs/core/datajoint-python/0.14/)